# TRABALHO PRÁTICO 1

- O trabalho prático deverá ser feito em dupla.
- A realização da entrega deverá ser feita via Teams, através da tarefa adicionada à equipe.
- Atente-se ao prazo de entrega, pois não será possível realizar a entrega após o prazo previsto.
- Apenas esse arquivo (`.ipynb`) com a resolução deverá ser entregue. Entregas em formato `.zip` serão penalizados!
- Apenas um aluno da dupla deverá fazer a entrega e colocar o nome da dupla.


## Informações da Dupla

| Nome         | Curso                  | Período | Turma |
| ------------ | ---------------------- | ------- | ----- |
| Fabio Neto   | Engenharia de Software | P7      | L1    |
| Davi Restani | Engenharia de Software | P7      | L1    |


---


## Problema de Buscas


### Instruções

Considere um tabuleiro quadrado onde cada bloco pode estar limpo (`'o'`) ou sujo (`'i'`).
Seu trabalho é mover o aspirador (`'x'`) pelo tabuleiro a fim de limpar todos os blocos sujos.

- Sempre que o aspirador deixa um bloco, este pode ser considerado como limpo.
- O objetivo é deixar todos espaços limpos (tabuleiro preenchido apenas com `'o'` e com o aspirador `'x'`)
- Para responder às questões, basta completar os códigos

<img src="images/problema.png" width = 150>


### Exemplo de Buscas

<img src="images/buscas.png" width = 1000>


## Implementação


### Bibliotecas

Importações necessárias para o funcionamento do código.


In [1]:
import numpy as np
from queue import Queue
from queue import LifoQueue
import heapq
import time


### Problema do Aspirador de Pó

Classe principal do problema. Heurística já adicionada.


In [2]:
class Aspirador():
    def __init__(self, tamanho):
        '''
        Construtor.

        Args:
            - tamanho: quantidade de linhas e colunas
        '''
        self.tamanho = tamanho

    def encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está.

        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.tamanho):
            for j in range(self.tamanho):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_objetivo(self, estado):
        '''
        Verifica se o estado atual é o objetivo.

        Objetivo:
            - Não haver sujeira ("i") -> Todos blocos, exceto onde o aspirador está devem ser "o".
        Args:
            - estado: estado atual do tabuleiro
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        item, cont = np.unique(estado, return_counts=True)
        mapa = dict()
        for i in range(len(item)):
            mapa[item[i]] = cont[i]
        # Todos elementos exceto onde o aspirador está
        if mapa['o'] == (self.tamanho**2 - 1):
            return True
        return False

    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados.

        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        novos_estados = []
        linha, coluna = self.encontra_posicao(atual, 'x')
        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1
            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'
            novos_estados.append(novo_estado)
        # Baixo
        if linha < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1
            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)
        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1
            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'
            novos_estados.append(novo_estado)
        # Direita
        if coluna < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1
            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'
            novos_estados.append(novo_estado)
        return novos_estados

    def heuristica(self, estado):
        '''
        Dado o estado atual, retorna o número de elementos sujos 'i'.

        Args:
            - estado: matriz contendo o estado do tabuleiro
        Return:
            - número de elementos sujos 'i'
        '''
        return np.count_nonzero(estado == 'i')


### Estratégias de Busca

Adicionada por motivos de organização.


In [3]:
class EstrategiaDeBusca():
    def __init__(self, problema):
        '''
        Construtor.

        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema

    def compara_estados(self, estado, estado_visitado):
        '''
        Compara dois estados.
        Caso haja alguma diferença, retorna False, senão retorna True.

        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True

    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verifica se um estado está na lista de visitados.

        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
        return False


### BFS

Busca em Largura.


In [4]:
class BreadthFirstSearch(EstrategiaDeBusca):
    def __init__(self, problema):
        EstrategiaDeBusca.__init__(self, problema)

    def busca(self, inicio):
        '''
        Realiza a busca BFS, armazenando os estados em uma FILA.

        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        fila = Queue()
        fila.put(inicio)
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        while not fila.empty():
            atual = fila.get()
            estados_visitados.append(atual)
            if self.problema.verifica_objetivo(atual):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self.verifica_visitados(i, estados_visitados):
                        fila.put(i)
        return solucao_encontrada, estados_visitados, cont_estados


### DFS

Busca em Profundidade.


In [5]:
class DepthFirstSearch(EstrategiaDeBusca):
    def __init__(self, problema):
        EstrategiaDeBusca.__init__(self, problema)

    def busca(self, inicio):
        '''
        Realiza a busca DFS, armazenando os estados em uma PILHA.

        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        pilha = LifoQueue()
        pilha.put(inicio)
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        while not pilha.empty():
            atual = pilha.get()
            estados_visitados.append(atual)
            if self.problema.verifica_objetivo(atual):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self.verifica_visitados(i, estados_visitados):
                        pilha.put(i)
        return solucao_encontrada, estados_visitados, cont_estados


### Busca Gulosa

Busca Gulosa.


In [6]:
class BuscaGulosa(EstrategiaDeBusca):
    def __init__(self, problema):
        EstrategiaDeBusca.__init__(self, problema)

    def busca(self, inicio):
        '''
        Realiza a busca gulosa, armazenando os estados em uma FILA DE PRIORIDADES.

        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados

        Obs.: A distância de Manhattan é inversamente proporcional à prioridade, quanto menor a distância, maior
        a prioridade desse estado.
        '''
        p_fila = []
        id_estado = 0
        heapq.heappush(p_fila, (0, id_estado, inicio))
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)
            if self.problema.verifica_objetivo(atual):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self.verifica_visitados(i, estados_visitados):
                        id_estado += 1
                        posicoes_sujas = self.problema.heuristica(i)
                        heapq.heappush(p_fila, (posicoes_sujas, id_estado, i))
        return solucao_encontrada, estados_visitados, cont_estados


### Função Auxiliar

Mostra os resultados alcançados.


In [7]:
def mostrar_resultados(resultados, tempo):
    solucao, _, num_visitados = resultados
    print(f'- Finalizado em {np.format_float_scientific(tempo)} segundos.')
    print(f'- Foram visitados {num_visitados} estados.')
    if solucao:
        print('- A solução foi encontrada! =)')
    else:
        print('- A solução não foi encontrada! =(')


## Testes

Executando três buscas com o tabuleiro inicial fornecido e comparando: o tempo de execução de cada busca, a quantidade de estados buscados(inclusive o estado objetivo) e se a solução foi encontrada ou não.


In [8]:
start = np.array([['x', 'i', 'o'],
                  ['i', 'i', 'i'],
                  ['o', 'o', 'i']])
problema = Aspirador(3)


In [9]:
# BFS
print('BFS:')
tempo = time.time()
bfs = BreadthFirstSearch(problema)
bfs_resultados = bfs.busca(start)
tempo_f = time.time() - tempo
mostrar_resultados(bfs_resultados, tempo_f)


BFS:
- Finalizado em 3.194999694824219e-02 segundos.
- Foram visitados 89 estados.
- A solução foi encontrada! =)


In [10]:
# DFS
print('DFS:')
tempo = time.time()
dfs = DepthFirstSearch(problema)
dfs_resultados = dfs.busca(start)
tempo = time.time() - tempo
mostrar_resultados(dfs_resultados, tempo)


DFS:
- Finalizado em 1.9953250885009766e-03 segundos.
- Foram visitados 9 estados.
- A solução foi encontrada! =)


In [11]:
# Busca Gulosa
print('Busca Gulosa:')
tempo = time.time()
bg = BuscaGulosa(problema)
bg_resultados = bg.busca(start)
tempo = time.time() - tempo
mostrar_resultados(bg_resultados, tempo)


Busca Gulosa:
- Finalizado em 1.0457038879394531e-03 segundos.
- Foram visitados 9 estados.
- A solução foi encontrada! =)
